# step 1


In [8]:
import pandas as pd
import json

# Read the CSV file
df = pd.read_csv('inputfiles/reliancenews.csv')

# Initialize the dictionary
data = {}

# Iterate over the DataFrame rows
for _, row in df.iterrows():
    date = str(row['Date'])
    headline = row['News']
    
    # Check if the date already exists in the dictionary
    if date in data:
        # Append the headline to the existing list
        data[date].append({"heading": headline})
    else:
        # Create a new list with the headline
        data[date] = [{"heading": headline,}]

# Open the JSON file in write mode
with open('reliancelstmheadlines.json', 'w', encoding='utf-8') as f:
    # Write the dictionary to the file
    json.dump(data, f, ensure_ascii=False)

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import json

In [18]:
modelName = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForSequenceClassification.from_pretrained(modelName)

In [19]:
def calculateDailySentiment(headlines):
    texts = [headline['heading'] for headline in headlines]
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512, return_attention_mask=True)
    outputs = model(**inputs)
    logits = outputs.logits
    scores = logits.softmax(dim=1)
    averageScore = scores.mean(dim=0).tolist()
    return averageScore
#enddef

def analyzeAndSaveSentiment(inputFile, outputFile):
    with open(inputFile, 'r',encoding='utf-8') as file:
        data = json.load(file)
    #endwith

    result = {}

    for date, headlines in data.items():
        averageScore = calculateDailySentiment(headlines)
        print(f"{date} > {averageScore}")
        result[date] = averageScore
    #endfor

    with open(outputFile, 'w') as outputFile:
        json.dump(result, outputFile, indent=2)
    #endwith
#enddef


In [20]:
inputJsonFile = 'reliancelstmheadlines.json'
outputJsonFile = 'reliancedaily_scores.json'

analyzeAndSaveSentiment(inputJsonFile, outputJsonFile)

03 Jan 2011 > [0.06955227255821228, 0.10576020181179047, 0.2655993103981018, 0.33184099197387695, 0.2272472381591797]
05 Jan 2011 > [0.05843456834554672, 0.08646899461746216, 0.22058098018169403, 0.33523881435394287, 0.29927659034729004]
07 Jan 2011 > [0.05296649783849716, 0.08330032229423523, 0.2243427336215973, 0.3453226387500763, 0.29406777024269104]
10 Jan 2011 > [0.07095428556203842, 0.09166199713945389, 0.2112600952386856, 0.3094114661216736, 0.3167121708393097]
11 Jan 2011 > [0.18488772213459015, 0.20468389987945557, 0.25550898909568787, 0.19920627772808075, 0.15571311116218567]
12 Jan 2011 > [0.08856333792209625, 0.13165563344955444, 0.28684893250465393, 0.2984969913959503, 0.19443513453006744]
13 Jan 2011 > [0.1391548067331314, 0.13974949717521667, 0.25078892707824707, 0.2510644793510437, 0.21924224495887756]
17 Jan 2011 > [0.017456959933042526, 0.035515401512384415, 0.18374350666999817, 0.40694674849510193, 0.35633736848831177]
19 Jan 2011 > [0.016322407871484756, 0.030849331